In [15]:
# Necessary packages
import pandas as pd
import opencc
import csv
import re

In [16]:
# Load uncleaned data
df_male = pd.read_csv("data/1976_raw_male.csv")
df_female = pd.read_csv("data/1976_raw_female.csv")
df_neutral = pd.read_csv("data/1976_raw_neutral.csv")

In [17]:
# Add a gender column
df_male['gender'] = 'male'
df_female['gender'] = 'female'
df_neutral['gender'] = 'neutral'

In [18]:
# Data merging
df = pd.concat([df_male, df_female, df_neutral], ignore_index=True)

In [19]:
# Translate Traditional Chinese into Simplified Chinese
converter = opencc.OpenCC('t2s.json')
for col in df.columns:
    if col == 'detail_url':
        continue
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: converter.convert(x) if isinstance(x, str) else x)

In [20]:
# Remove escape characters from the description column
if 'description' in df.columns:
    df['description'] = df['description'].replace({r'\r\n|\n|\r': ''}, regex=True)

In [21]:
# Delete records with empty notes or fragrance
print(df[['fragrance', 'top_notes', 'middle_notes', 'base_notes']].isna().sum())
df = df.dropna(subset=['fragrance', 'top_notes', 'middle_notes', 'base_notes'])

fragrance       110
top_notes       155
middle_notes    301
base_notes      314
dtype: int64


In [22]:
# Remove extra spaces in the notes and fragrance
for col in ['fragrance', 'top_notes', 'middle_notes', 'base_notes']:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

In [23]:
# Choose the first fragrance as classification
df['fragrance'] = df['fragrance'].apply(lambda x: x.split('、')[0] if isinstance(x, str) else x)

In [24]:
# Count the number of notes
note_columns = ['top_notes', 'middle_notes', 'base_notes']
note_counts = {}

for col in note_columns:
    if col in df.columns:
        for s in df[col]:
            if isinstance(s, str):
                for item in [n.strip() for n in s.split('、') if n.strip()]:
                    note_counts[item] = note_counts.get(item, 0) + 1

# Save
with open('data/note_count.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['note', 'count'])
    for item, count in sorted(note_counts.items(), key=lambda x: x[1], reverse=True):
        writer.writerow([item, count])

In [25]:
# Count the number of fragrance
fragrance_counts = {}
for s in df['fragrance']:
    for item in [i.strip() for i in s.split('、') if i.strip()]:
        fragrance_counts[item] = fragrance_counts.get(item, 0) + 1

# Save
with open('data/fragrance_count.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['fragrance', 'count'])
    for item, count in sorted(fragrance_counts.items(), key=lambda x: x[1], reverse=True):
        writer.writerow([item, count])

In [26]:
# Unified fragrance label
df = df.rename(columns={'fragrance': 'original_fragrance'})
map_fragrance = pd.read_csv("data/fragrance_map.csv") # Manually unify the expression of fragrance labels
df = df.merge(map_fragrance, how='left', left_on='original_fragrance', right_on='original_fragrance')

In [27]:
# Extract brand from the name
def extract_brand_optimized(product_name):
    if pd.isna(product_name):
        return pd.NA
    
    product_name_str = str(product_name).strip()
    
    brand_dictionary = {
        # Luxury Brands
        'YSL': ['YSL', 'Yves Saint Laurent', '伊夫·圣罗兰', '圣罗兰'],
        'Versace': ['Versace', 'Versace Collection', '范思哲', '凡赛斯'],
        'Dior': ['Dior', 'Christian Dior', '迪奥'],
        'Chanel': ['Chanel', 'Gabrielle Chanel', '香奈儿'],
        'Gucci': ['Gucci', 'Gucci Beauty', '古驰'],
        'Hermes': ['Hermes', 'Hermès Parfums', '爱马仕'],
        'Louis Vuitton': ['Louis Vuitton', 'LV', 'Louis Vuitton Parfums', '路易威登', '驴牌'],
        'Givenchy': ['Givenchy', 'Givenchy Beauty', '纪梵希'],
        'Burberry': ['Burberry', 'Burberry Beauty', '博柏利', '巴宝莉'],
        'Dolce & Gabbana': ['Dolce & Gabbana', 'D&G', 'Dolce Gabbana', '杜嘉班纳'],
        'Valentino': ['Valentino', 'Valentino Beauty', '华伦天奴'],
        'Armani': ['Armani', 'Giorgio Armani', 'Armani Beauty', '阿玛尼'],
        'Prada': ['Prada', 'Prada Beauty', '普拉达'],
        'Fendi': ['Fendi', 'Fendi Parfums', '芬迪'],
        'Bottega Veneta': ['Bottega Veneta', 'Bottega', 'BV', '葆蝶家'],
        'Celine': ['Celine', 'Celine Parfums', '思琳'],
        'Saint Laurent': ['Saint Laurent', 'YSL Saint Laurent', '圣罗兰（独立线）'],
        'Lanvin': ['Lanvin', 'Lanvin Parfums', '浪凡'],
        'Bvlgari': ['Bvlgari', 'Bulgari', 'BVLGARI Parfums', '宝格丽'],
        'Cartier': ['Cartier', 'Cartier Parfums', '卡地亚'],
        
        # High-End Niche Brands
        'Creed': ['Creed', 'Creed Fragrances', '克雷德'],
        'Byredo': ['Byredo', 'Byredo Parfums', '百瑞德'],
        'Le Labo': ['Le Labo', 'Le Labo Fragrances', '勒拉博'],
        'Diptyque': ['Diptyque', 'Diptyque Paris', 'Diptyque Do Son', 'Diptyque Philosykos', '蒂普提克'],
        'Serge Lutens': ['Serge Lutens', 'Serge Lutens Parfums', 'Serge Lutens Femme', 'Serge Lutens Homme', '芦丹氏', '卤蛋'],
        'Maison Margiela': ['Maison Margiela', 'MMM', 'Margiela', 'Maison Margiela Replica', 'Margiela Replica', '马丁·马吉拉', '马吉拉'],
        'Acqua di Parma': ['Acqua di Parma', 'Acqua di Parma Colonia', '帕尔玛之水'],
        'Penhaligon\'s': ['Penhaligon\'s', 'Penhaligon\'s London', '潘海利根'],
        'Roja Parfums': ['Roja Parfums', 'Roja Dove', '罗嘉德芬'],
        'Amouage': ['Amouage', 'Amouage Parfums', '爱慕'],
        'Tom Ford': ['Tom Ford', 'Tom Ford Beauty', 'TF', '汤姆·福特'],
        'Jo Malone': ['Jo Malone', 'Jo Malone London', 'JM', '祖玛珑', '祖马龙'],
        'Atelier Cologne': ['Atelier Cologne', 'Atelier Cologne Parfums', '欧珑'],
        'Maison Francis Kurkdjian': ['Maison Francis Kurkdjian', 'MFK', 'Francis Kurkdjian', '弗朗西斯·库尔吉安'],
        'Clive Christian': ['Clive Christian', 'Clive Christian Parfums', '克莱夫·克里斯蒂安'],
        'Ormonde Jayne': ['Ormonde Jayne', 'Ormonde Jayne London', '欧梦德·杰尼'],
        'Frederic Malle': ['Frederic Malle', 'Frederic Malle Parfums', '弗雷德里克·马勒'],
        'Parfums de Marly': ['Parfums de Marly', 'Marly', '玛丽之香'],
        'Kilian': ['Kilian', 'Kilian Hennessy', 'Kilian Parfums', 'By Kilian', 'Kilian Paris', '基利安'],
        'Memo Paris': ['Memo Paris', 'Memo', '记忆香水'],
        
        # Mass Market Brands
        'Calvin Klein': ['Calvin Klein', 'CK', 'CK Beauty', '卡尔文·克莱恩'],
        'Ralph Lauren': ['Ralph Lauren', 'Polo Ralph Lauren', 'RL', '拉夫·劳伦', '保罗·劳伦'],
        'DKNY': ['DKNY', 'Donna Karan New York', '唐可娜儿'],
        'Coach': ['Coach', 'Coach Fragrances', '蔻驰'],
        'Kate Spade': ['Kate Spade', 'Kate Spade New York', '凯特·丝蓓'],
        'Marc Jacobs': ['Marc Jacobs', 'Marc Jacobs Beauty', 'MJ', '马克·雅可布'],
        'Tory Burch': ['Tory Burch', 'Tory Burch Beauty', '托里·伯奇'],
        'Vera Wang': ['Vera Wang', 'Vera Wang Fragrances', '王薇薇'],
        'Michael Kors': ['Michael Kors', 'MK', 'Michael Kors Beauty', '迈克高仕'],
        'Guess': ['Guess', 'Guess Fragrances', '盖尔斯'],
        'Abercrombie & Fitch': ['Abercrombie & Fitch', 'A&F', 'Abercrombie', '阿贝克隆比 & 费奇'],
        'Hollister': ['Hollister', 'Hollister Co.', '好利斯特'],
        'Tommy Hilfiger': ['Tommy Hilfiger', 'Tommy', 'Tommy Hilfiger Fragrances', '汤米·希尔费格'],
        'Nautica': ['Nautica', 'Nautica Fragrances', '诺帝卡'],
        'Lacoste': ['Lacoste', 'Lacoste Fragrances', '鳄鱼'],
        'Hugo Boss': ['Hugo Boss', 'Boss', 'Hugo', '雨果博斯', '波士'],
        'Aramis': ['Aramis', 'Aramis Beauty', '雅男士'],
        'Davidoff': ['Davidoff', 'Davidoff Fragrances', '大卫杜夫'],
        'Dunhill': ['Dunhill', 'Alfred Dunhill', 'Dunhill Parfums', '登喜路'],
        'Montblanc': ['Montblanc', 'Montblanc Fragrances', '万宝龙'],
        
        # Designer Brands
        'Kenzo': ['Kenzo', 'Kenzo Parfums', 'Kenzo Takada', '高田贤三', '凯卓'],
        'Issey Miyake': ['Issey Miyake', 'Issey Miyake Parfums', '三宅一生'],
        'Loewe': ['Loewe', 'Loewe Parfums', '罗意威'],
        'Comme des Garçons': ['Comme des Garçons', 'CDG', 'Comme des Garçons Parfums', '川久保玲'],
        'Jil Sander': ['Jil Sander', 'Jil Sander Parfums', '吉尔·桑达'],
        'Helmut Lang': ['Helmut Lang', 'Helmut Lang Fragrances', '海尔姆特·朗'],
        'Annick Goutal': ['Annick Goutal', 'Annick Goutal Parfums', '安霓可·古特尔'],
        
        # Niche & Specialized Brands
        'BDK Parfums': ['BDK Parfums', 'BDK', 'BDK香水'],
        'Miller Et Bertaux': ['Miller Et Bertaux', 'Miller & Bertaux', '米勒贝尔托'],
        'Bentley': ['Bentley', 'Bentley Fragrances', '宾利'],
        'Bugatti': ['Bugatti', 'Bugatti Fragrances', '布加迪'],
        'Ferrari': ['Ferrari', 'Ferrari Fragrances', '法拉利'],
        'Porsche Design': ['Porsche Design', 'Porsche Design Fragrances', '保时捷设计'],
        'Jaguar': ['Jaguar', 'Jaguar Fragrances', '捷豹'],
        'Lamborghini': ['Lamborghini', 'Lamborghini Fragrances', '兰博基尼'],
        'Chloe': ['Chloe', 'Chloe Parfums', '克洛伊'],
        'See By Chloe': ['See By Chloe', 'See By Chloe Fragrances', '副线克洛伊'],
        'Stella McCartney': ['Stella McCartney', 'Stella McCartney Parfums', '斯特拉·麦卡特尼'],
        'Balenciaga': ['Balenciaga', 'Balenciaga Parfums', '巴黎世家'],
        'Alexander McQueen': ['Alexander McQueen', 'Alexander McQueen Parfums', '亚历山大·麦昆'],
        'Viktor & Rolf': ['Viktor & Rolf', 'Viktor Rolf', '维果罗夫'],
        'Jean Paul Gaultier': ['Jean Paul Gaultier', 'JPG', 'Jean Paul Gaultier Parfums', '高缇耶'],
        'Thierry Mugler': ['Thierry Mugler', 'Mugler', 'Mugler Parfums', '穆勒'],
        'Azzaro': ['Azzaro', 'Azzaro Parfums', '阿莎罗'],
        'Fragonard': ['Fragonard', 'Fragonard Parfums', '花宫娜'],
        'L\'Artisan Parfumeur': ['L\'Artisan Parfumeur', 'L\'Artisan', '阿蒂仙之香'],
        
        # Celebrity & Crossover Brands
        'Messi': ['Messi', 'Messi Fragrances', '梅西'],
        'David Beckham': ['David Beckham', 'DB', 'David Beckham Fragrances', '大卫·贝克汉姆'],
        'Jennifer Lopez': ['Jennifer Lopez', 'J.Lo', 'Jennifer Lopez Fragrances', '詹妮弗·洛佩兹'],
        'Britney Spears': ['Britney Spears', 'Britney', 'Britney Spears Fragrances', '布兰妮'],
        'Beyoncé': ['Beyoncé', 'Beyoncé Fragrances', '碧昂斯'],
        'Rihanna': ['Rihanna', 'Fenty Beauty', 'Rihanna Fragrances', '蕾哈娜'],
        'Justin Bieber': ['Justin Bieber', 'Justin Bieber Fragrances', '贾斯汀·比伯'],
        'One Direction': ['One Direction', '1D', 'One Direction Fragrances', '单向组合'],
        'Taylor Swift': ['Taylor Swift', 'Taylor Swift Fragrances', '泰勒·斯威夫特'],
        'Katy Perry': ['Katy Perry', 'Katy Perry Fragrances', '凯蒂·佩里']
    }
    
    for brand, aliases in brand_dictionary.items():
        for alias in aliases:
            if alias in product_name_str and (
                product_name_str.startswith(alias) or f' {alias}' in product_name_str
            ):
                return brand
    
    match = re.match(r'^([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)', product_name_str)
    if match:
        potential_brand = match.group(1).strip()
        non_brand_words = ['The', 'A', 'An', 'For']
        if len(potential_brand) > 1 and potential_brand not in non_brand_words:
            if not re.search(r'[0-9()（）]', potential_brand):
                return potential_brand
    
    return pd.NA

df['brand'] = df['name'].apply(extract_brand_optimized)

In [28]:
# Save the final cleaned data
df.to_csv("data/1976_clean.csv",index=False)